In [ ]:
import numpy as np
import pandas as pd
import math

# Off5th data cleaning

In [ ]:
off5th_raw = pd.read_csv('off5th.csv')

In [ ]:
off5th_raw.head()

In [ ]:
off5th_raw['category'].value_counts()

In [ ]:
off5th_raw.loc[off5th_raw['category']=='dress-shirts','order'] = (range(1,np.sum(off5th_raw['category']=='dress-shirts')+1))
off5th_raw.loc[off5th_raw['category']=='dresses','order'] = (range(1,np.sum(off5th_raw['category']=='dresses')+1))
off5th_raw.loc[off5th_raw['category']=='casual-button-down-shirts','order'] = (range(1,np.sum(off5th_raw['category']=='casual-button-down-shirts')+1))                                                       
off5th_raw.loc[:,'order'] = off5th_raw['order'].astype(int)

In [ ]:
np.sum(off5th_raw.isnull())

In [ ]:
off5th_raw.shape

In [ ]:
off5th_raw.loc[off5th_raw['original_price']<off5th_raw['discount_price'],'original_price'] = off5th_raw.loc[off5th_raw['original_price']<off5th_raw['discount_price'],'discount_price']

## Add percent_off column

In [ ]:
off5th_raw.loc[:,'percent_off'] = round((1-off5th_raw['discount_price']/off5th_raw['original_price'])*100,2)

In [ ]:
off5th_dups = off5th_raw.loc[off5th_raw.duplicated(['brand_name','product_name','original_price','discount_price'])]

In [ ]:
off5th_dups['brand_name']

In [ ]:
off5th_raw[['discount_amount','percent_off']].describe()

### Discount amount marked on off5th.com is about 1% lower than calculated percentage off on displayed price. Possibly due to on page rounding methods.

In [ ]:
off5th_raw.loc[(off5th_raw['percent_off']>0)][['discount_amount','percent_off']].describe()

In [ ]:
off5th_raw['brand_name'].value_counts()

## Add private_label column

In [ ]:
off_private_labels = ['Cashmere Saks Fifth Avenue','Saks Fifth Avenue',
                      'Saks Fifth Avenue BLACK', 'Saks Fifth Avenue Collection', 
                      'Saks Fifth Avenue Made In Italy','Saks Fifth Avenue OFF 5TH',
                      'Saks Fifth Avenue Travel',
                      'Pure Navy']

In [ ]:
off5th_raw.loc[off5th_raw['brand_name'].isin(off_private_labels),'private_label'] = 'Private Label'

In [ ]:
off5th_raw.loc[~off5th_raw['brand_name'].isin(off_private_labels),'private_label'] = 'Non-Private'

In [ ]:
off5th_raw['private_label'].value_counts()

In [ ]:
off5th_raw.sample(10)

## Add 'brand_positioning' column
### Same brands will directly use the result from Saks.com, other brands will be selected again by the same standard

In [ ]:
saks = pd.read_csv('saks_processed.csv')

In [ ]:
saks.head()

In [ ]:
saks_brand = saks.loc[~saks[['brand_name','brand_mean','brand_positioning']].duplicated()][['brand_name','brand_mean','brand_positioning']]
saks_brand

In [ ]:
offth_partial = pd.merge(off5th_raw, saks_brand,how='left',on='brand_name')

In [ ]:
off5th_men = offth_partial.loc[(offth_partial['department']=='men') & (offth_partial['brand_positioning'].isna())]
off5th_women = offth_partial.loc[(offth_partial['department']=='women') & (offth_partial['brand_positioning'].isna())]
unisex_brands = list(set.intersection(set(off5th_men['brand_name'].unique()),
                                      set(off5th_women['brand_name'].unique())))

In [ ]:
off5th_men.head()

In [ ]:
off5th_uni = offth_partial.loc[offth_partial['brand_name'].isin(unisex_brands)]
off5th_men = offth_partial.loc[(offth_partial['department']=='men') 
                               & (~offth_partial['brand_name'].isin(unisex_brands)) 
                               & (offth_partial['brand_positioning'].isna())]
off5th_women = offth_partial.loc[(offth_partial['department']=='women') 
                              & (~offth_partial['brand_name'].isin(unisex_brands)) 
                              & (offth_partial['brand_positioning'].isna())]

In [ ]:
off5th_uni

In [ ]:
men_brand_price = off5th_men.groupby('brand_name').mean()[['original_price']].sort_values(by='original_price')
women_brand_price = off5th_women.groupby('brand_name').mean()[['original_price']].sort_values(by='original_price')
uni_brand_price = off5th_uni.groupby('brand_name').mean()[['original_price']].sort_values(by='original_price')

In [ ]:
men_brand_price.reset_index()

In [ ]:
men_brand_price.loc[men_brand_price['original_price']<=150,'brand_positioning'] = 'Trendy'
men_brand_price.loc[(men_brand_price['original_price']>150) & (men_brand_price['original_price']<400),'brand_positioning'] = 'Contemporary'
men_brand_price.loc[men_brand_price['original_price']>=400,'brand_positioning'] = 'Luxury'

In [ ]:
men_brand_price[men_brand_price['brand_positioning'] == 'Luxury']

In [ ]:
women_brand_price

In [ ]:
women_brand_price.loc[women_brand_price['original_price']<=200,'brand_positioning'] = 'Trendy'
women_brand_price.loc[(women_brand_price['original_price']>200) & (women_brand_price['original_price']<1000),'brand_positioning'] = 'Contemporary'
women_brand_price.loc[women_brand_price['original_price']>=1000,'brand_positioning'] = 'Luxury'

In [ ]:
women_brand_price['brand_positioning'].value_counts()

In [ ]:
uni_brand_price.describe()

In [ ]:
uni_brand_price.loc[uni_brand_price['original_price']<=320,'brand_positioning'] = 'Trendy'
uni_brand_price.loc[(uni_brand_price['original_price']>320) & (uni_brand_price['original_price']<800),'brand_positioning'] = 'Contemporary'
uni_brand_price.loc[uni_brand_price['original_price']>=800,'brand_positioning'] = 'Luxury'

In [ ]:
uni_brand_price

In [ ]:
offth_partial[(offth_partial['brand_name']=='Saks Fifth Avenue')|(offth_partial['brand_name']=='Donna Karan')].sample(5)

In [ ]:
partpart = offth_partial.loc[offth_partial['brand_positioning'].isna()][['website','brand_name','product_name','category','department','original_price','discount_price','discount_amount','order','percent_off','private_label']]
partpart.head()
saks_part = offth_partial.loc[~offth_partial['brand_positioning'].isna()]
saks_part.head()

In [ ]:
off5th_men = pd.merge(partpart, men_brand_price.reset_index(), on='brand_name')
off5th_women = pd.merge(partpart, women_brand_price.reset_index(), on='brand_name')
off5th_uni = pd.merge(partpart, uni_brand_price.reset_index(), on='brand_name')

In [ ]:
off5th_men.head()

In [ ]:
part_full = pd.concat([off5th_men,off5th_women,off5th_uni],ignore_index=True)

In [ ]:
part_full.sample(10)

In [ ]:
part_full.columns

In [ ]:
part_full = part_full.rename(columns={'original_price_y':'brand_mean','original_price_x':'original_price'})

In [ ]:
off5th_full = pd.concat([saks_part,part_full],ignore_index=True)

In [ ]:
np.sum(off5th_full.isna())

In [ ]:
off5th_full['brand_positioning'].value_counts()

## EDA

In [ ]:
for feature in ['original_price','discount_price','percent_off','order']:
    print(off5th_full.groupby('brand_positioning').agg(['mean','std','min','max'])[[feature]])

In [ ]:
for feature in ['original_price','discount_price','percent_off']:
    print(off5th_full.groupby('category').agg(['mean','std','min','max'])[[feature]])

In [ ]:
off5th_full.groupby(['category','brand_positioning']).agg(['count','mean','std','min','max'])[['original_price']]

In [ ]:
off5th_full.groupby(['category','brand_positioning']).agg(['count','mean','std','min','max'])[['discount_price']]

In [ ]:
off5th_full.groupby(['category','brand_positioning']).agg(['count','mean','std','min','max'])[['percent_off']]

## Visualization

In [ ]:
import plotly
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
import plotly.figure_factory as ff

contemporary = off5th_full.loc[off5th_full['brand_positioning']=='Contemporary']['original_price']
trendy = off5th_full.loc[off5th_full['brand_positioning']=='Trendy']['original_price']
luxury = off5th_full.loc[off5th_full['brand_positioning']=='Luxury']['original_price']

hist_data = [trendy, contemporary, luxury]
group_labels = ['Trendy', 'Contemporary', 'Luxury']
colors = ['rgb(25, 43, 87)', 'rgb(240, 188, 84)', 'rgb(166, 164, 161)']

fig = ff.create_distplot(hist_data, group_labels, bin_size=75, colors= colors)
fig.update_layout(title_text='Items Original Price Distribution On Off5th.com By Brand Type',plot_bgcolor='rgb(256, 256, 256)')
fig.update_xaxes(title_text = "Price(USD)")
fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='rgb(230, 230, 230)')

fig.show()

In [ ]:
import plotly.express as px
colors = ['rgb(25, 43, 87)', 'rgb(201, 161, 91)', 'rgb(166, 164, 161)']
fig = px.box(off5th_full, x="brand_positioning", y="original_price", points=False, 
             color_discrete_sequence = ['rgb(25, 43, 87)','rgb(240, 188, 84)', 'rgb(166, 164, 161)'],
             color_discrete_map = {"brand_positioning": ["Trendy", "Contemporary", "Luxury"]})
#              color_discrete_map={'Trendy':'rgb(25, 43, 87)',
#                                  'Contemporary':'rgb(201, 161, 91)',
#                                  'Luxury':'rgb(166, 164, 161)'
#                                  })

fig.update_layout(title_text='Price By Brand Type',plot_bgcolor='rgb(256, 256, 256)')
fig.update_xaxes(title_text = "")
# fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='rgb(230, 230, 230)')
fig.show()

In [ ]:
import plotly.figure_factory as ff

contemporary = off5th_full.loc[off5th_full['brand_positioning']=='Contemporary']['discount_price']
trendy = off5th_full.loc[off5th_full['brand_positioning']=='Trendy']['discount_price']
luxury = off5th_full.loc[off5th_full['brand_positioning']=='Luxury']['discount_price']

hist_data = [trendy, contemporary, luxury]
group_labels = ['Trendy', 'Contemporary', 'Luxury']
colors = ['rgb(25, 43, 87)', 'rgb(240, 188, 84)', 'rgb(166, 164, 161)']

fig = ff.create_distplot(hist_data, group_labels, bin_size=20, colors= colors)
fig.update_layout(title_text='Items Discount Price Distribution On Off5th.com By Brand Type',plot_bgcolor='rgb(256, 256, 256)')
fig.update_xaxes(title_text = "Price(USD)") 
fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='rgb(230, 230, 230)')

fig.show()

In [ ]:
colors = ['rgb(25, 43, 87)', 'rgb(201, 161, 91)', 'rgb(166, 164, 161)']
fig = px.box(off5th_full, x="brand_positioning", y="discount_price", points='outliers', 
             color_discrete_sequence = ['rgb(25, 43, 87)','rgb(240, 188, 84)', 'rgb(166, 164, 161)'],
             color_discrete_map = {"brand_positioning": ["Trendy", "Contemporary", "Luxury"]})
#              color_discrete_map={'Trendy':'rgb(25, 43, 87)',
#                                  'Contemporary':'rgb(201, 161, 91)',
#                                  'Luxury':'rgb(166, 164, 161)'
#                                  })

fig.update_layout(title_text='Price By Brand Type',plot_bgcolor='rgb(256, 256, 256)')
fig.update_xaxes(title_text = "")
fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='rgb(230, 230, 230)')
fig.show()

In [ ]:
off5th_melt = pd.melt(off5th_full,id_vars=['brand_name','product_name','category','brand_positioning','private_label'],value_vars=['original_price','discount_price','percent_off'])

In [ ]:
import seaborn as sns

sns.set_style("whitegrid")

ax = sns.boxplot(x="brand_positioning", y="value", hue="variable", 
                 data=off5th_melt.loc[off5th_melt['variable']!='percent_off'], 
                 palette=['#192b57','#a6a4a1'])
ax.set(ylim=(0, 4000))
ax.set(ylabel='',xlabel='',title = 'Clothing Average Price on Off5th.com By Brand Type')

In [ ]:
fig = ax.get_figure()
fig.savefig("avg_off5_price.png")
fig.savefig("avg_off5_price.pdf")

In [ ]:
import plotly.express as px

fig = px.parallel_categories(off5th_full.iloc[:,1:], color="percent_off", color_continuous_scale=px.colors.sequential.Cividis)
fig.show()

In [ ]:
import plotly.express as px

fig = px.scatter(off5th_full, x="original_price", y="percent_off", color="brand_positioning",
                 hover_data=['brand_name'], 
                 color_discrete_sequence = ['rgb(25, 43, 87)','rgb(240, 188, 84)', 'rgb(166, 164, 161)'],
                 color_discrete_map = {"brand_positioning": ["Trendy", "Contemporary", "Luxury"]})
fig.update_layout(title_text="Scatter Plot for Original Price and Discount Percent",plot_bgcolor='rgb(256, 256, 256)')
fig.update_yaxes(title_text = 'Discount Percent',showgrid=True, gridwidth=0.5, gridcolor='rgb(230, 230, 230)')
fig.update_xaxes(title_text = "Price(USD)") 

fig.show()

In [ ]:
off5th_full['brand_positioning'].value_counts()

In [ ]:
import plotly.graph_objects as go

labels = ['Trendy','Contemporary','Luxury']
values = [1655, 2779, 677]
colors = ['rgb(25, 43, 87)','rgb(240, 188, 84)','rgb(220, 220, 220)']
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(hoverinfo='label+value', textinfo='percent',
                  marker=dict(colors=colors))
fig.show()

In [ ]:
off5th_full['private_label'].value_counts()

In [ ]:
import plotly.graph_objects as go

labels = ['Private Label','Non-Private']
values = [352, 4759]
colors = ['rgb(25, 43, 87)','rgb(220, 220, 220)'] #'rgb(201, 161, 91)',
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(hoverinfo='label+value', textinfo='percent',
                  marker=dict(colors=colors))
fig.show()

In [ ]:
off5th_full['category'].value_counts()

In [ ]:
import plotly.graph_objects as go

labels = ['dresses','casual-button-down-shirts','dress-shirts']
values = [3863, 632, 616]
colors = ['rgb(247, 146, 171)','rgb(173, 245, 100)','rgb(167, 197, 209)']
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(hoverinfo='label+value', textinfo='percent',
                  marker=dict(colors=colors))
fig.show()

In [ ]:
colors = ['rgb(247, 146, 171)','rgb(173, 245, 100)','rgb(167, 197, 209)']
labels = ['dresses','casual-button-down-shirts']
fig = px.box(off5th_full, x="category", y="original_price", points='outliers', 
             color_discrete_sequence = colors,
#              color_discrete_map = {"category": labels})
             color_discrete_map={'Trendy':'rgb(25, 43, 87)',
                                 'Contemporary':'rgb(201, 161, 91)',
                                 'Luxury':'rgb(166, 164, 161)'
                                 })

fig.update_layout(title_text='Price By Category',plot_bgcolor='rgb(256, 256, 256)')
fig.update_xaxes(title_text = "")
fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='rgb(230, 230, 230)')
fig.show()

In [ ]:
off5th_full.head()

In [ ]:
off5th_full.to_csv('off5th_processed.csv',index=False)